In [4]:
#理解层和块
import torch
from torch import nn
from torch.nn import functional as F

#使用pytorch框架中的nn定义一个输入层为20个单元，一个隐含层为256个单元，输出层为10个单元的MLP，隐含层的激活函数为Relu
net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X = torch.rand(2,20)

In [5]:
net(X) #因为net是nn中的一个神经网络块（module）的实例，可以直接传入X表示进行前向传播，得到输出

tensor([[-0.1596, -0.1574,  0.0170,  0.1965, -0.0103, -0.0279, -0.0447, -0.0112,
         -0.1234,  0.0039],
        [ 0.0224, -0.1651, -0.0117,  0.2612, -0.0365, -0.0135, -0.0992, -0.0723,
          0.0065,  0.1121]], grad_fn=<AddmmBackward0>)

In [7]:
#自定义块，更深理解块以及块是如何工作的。
#每个块的基本功能：1.将输入数据作为其前向传播函数的参数。2.通过前向传播生成输出。3.计算输出关于输入的梯度。4.存储和访问前向传播所需的参数。5.根据需要初始化模型参数
class MLP(nn.Module):#MLP继承nn.Module类
    def __init__(self):
        #调用父类Module的构造函数进行必要的初始化
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    #定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [8]:
net1 = MLP()
net1(X)#自动调用forward函数进行前向传播

tensor([[ 0.0668,  0.1403, -0.0779,  0.2555, -0.1375, -0.0117,  0.1649,  0.0607,
          0.0568,  0.1540],
        [ 0.1014, -0.0037, -0.1562,  0.3076, -0.1183, -0.0377,  0.0505, -0.0546,
          0.0184,  0.1502]], grad_fn=<AddmmBackward0>)

In [11]:
#自定Sequential类，该类用于将其他模块串起来
#该类有两个关键函数,1.将块逐个追加到列表中的函数，2.一种前向传播函数，用于将输入按追加块顺序传递给块组成的链条
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):
            #这里，module是Module子类的一个实例，我们把它保存在‘Module’类的成员变量_modules中。_module的类型是OrderedDict，按字典形式存放各个块
            self._modules[str(idx)] = module

    def forward(self,X):
        '''实现前向传播'''
        # OrderedDict保证了按成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

#使用MySequentia类实现多层感知机
net2 = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net2(X)

tensor([[-0.1342,  0.1439,  0.0455,  0.1240, -0.0530, -0.1770,  0.0934,  0.1392,
          0.0214, -0.1790],
        [ 0.0846,  0.1677,  0.0049,  0.0899, -0.1369, -0.1382,  0.2171,  0.1469,
          0.0320, -0.1981]], grad_fn=<AddmmBackward0>)

In [12]:
#我们可以混合搭配各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)

    def forward(self,X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(),nn.Linear(16,32))
chimera(X)

tensor([[-0.1409,  0.0509,  0.1279, -0.2746, -0.1020,  0.0554, -0.2457, -0.0703,
         -0.1428,  0.2316,  0.2884, -0.1731,  0.1598, -0.0839,  0.2136, -0.1324,
          0.1637, -0.2109,  0.0449, -0.1455, -0.3685,  0.0544,  0.3265, -0.0363,
         -0.2703, -0.0048, -0.1511,  0.0050,  0.1955,  0.3132,  0.0814,  0.3281],
        [-0.0853,  0.0535,  0.1525, -0.2590, -0.1071,  0.0434, -0.2235, -0.0691,
         -0.1454,  0.2175,  0.2570, -0.1848,  0.1197, -0.1088,  0.2151, -0.1260,
          0.1641, -0.1865,  0.0557, -0.1729, -0.3381,  0.0307,  0.2845, -0.0377,
         -0.2397,  0.0055, -0.1345, -0.0017,  0.2214,  0.3198,  0.0686,  0.3261]],
       grad_fn=<AddmmBackward0>)